# Parameters of a Gaussian
In this notebook I fit some data to find the parameters of a Gaussian used to generate that data. This is all pulled completely from Kyle Lo's post found here: http://kyleclo.github.io/maximum-likelihood-in-tensorflow-pt-1/.

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Set the true parameters
TRUE_MU     = 10.
TRUE_SIGMA  = 5.0
SAMPLE_SIZE = 100

In [3]:
#Make the training data
np.random.seed(0)
x_obs = np.random.normal(loc  = TRUE_MU,
                        scale = TRUE_SIGMA,
                        size  = SAMPLE_SIZE)

In [4]:
#Rescale sub-obtimally. In reality one would use
#the mean and standard deviation
CENTER = x_obs.min()
SCALE  = x_obs.max() - x_obs.min()
x_obs = (x_obs - CENTER) / SCALE

Below, we swtich from working with $\sigma$ to working with the variance $\phi=\sigma^2$. This is to prevent the optimizer from going to negative standard deviations, but amounts to putting a prior on $\sigma$.

In [5]:
#Define a PGM
#Placeholder for the data
x = tf.placeholder(dtype=tf.float32)

INIT_MU_PARAMS  = {'loc': 0.0, 'scale':1.0}
INIT_PHI_PARAMS = {'loc': 0.0, 'scale':1.0}
RANDOM_SEED = 0

#Parameters
mu  = tf.Variable(initial_value=np.random.normal(**INIT_MU_PARAMS),
                  dtype=tf.float32)
phi = tf.Variable(initial_value=np.random.normal(**INIT_PHI_PARAMS),
                  dtype=tf.float32)
sigma = tf.square(phi)

In [6]:
#Define the loss function,
gaussian_dist = tf.contrib.distributions.Normal(loc=mu, scale=sigma)
log_prob = gaussian_dist.log_prob(value=x)
neg_log_likelihood = -1.0 * tf.reduce_sum(log_prob)

grad = tf.gradients(neg_log_likelihood, [mu, phi])

In [7]:
#Set up the optimization scheme
LEARNING_RATE = 0.001
MAX_ITER = 10000
TOL_PARAM, TOL_LOSS, TOL_GRAD = 1e-8, 1e-8, 1e-8

In [8]:
optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
train_op  = optimizer.minimize(loss=neg_log_likelihood)

In [9]:
with tf.Session() as sess:
    sess.run(fetches=tf.global_variables_initializer())
    
    i = 1
    obs_mu, obs_phi, obs_sigma = sess.run(fetches=[[mu], [phi], [sigma]])
    obs_loss = sess.run(fetches=[neg_log_likelihood], feed_dict={x: x_obs})
    obs_grad = sess.run(fetches=[grad], feed_dict={x: x_obs})
    
    while True:
        sess.run(fetches=train_op, feed_dict={x: x_obs})
        
        #update parameters
        new_mu, new_phi, new_sigma = sess.run(fetches=[mu, phi, sigma])
        diff_norm = np.linalg.norm(np.subtract([new_mu, new_phi],
                                              [obs_mu[-1], obs_phi[-1]]))
        #update loss
        new_loss = sess.run(fetches=neg_log_likelihood, feed_dict={x: x_obs})
        loss_diff = np.abs(new_loss - obs_loss[-1])
        
        #update gradient
        new_grad = sess.run(fetches=grad, feed_dict={x: x_obs})
        grad_norm = np.linalg.norm(new_grad)
        
        obs_mu.append(new_mu)
        obs_phi.append(new_phi)
        obs_sigma.append(new_sigma)
        obs_loss.append(new_loss)
        obs_grad.append(new_grad)
        
        if diff_norm < TOL_PARAM:
            print('Parameter convergence in {} iterations.'.format(i))
            break
        if loss_diff < TOL_LOSS:
            print('Loss function convergence in {} iterations.'.format(i))
            break
        if grad_norm < TOL_GRAD:
            print('Gradient convergence in {} iterations.'.format(i))
            break
        if i >= MAX_ITER:
            print('Max iterations reached without convergence.')
            break
        i += 1

Loss function convergence in 1202 iterations.


In [10]:
final_mu = obs_mu[-1]
final_sigma = obs_sigma[-1]
print("Final parameters:",final_mu, final_sigma)
print("True parameters: ",np.mean(x_obs), np.std(x_obs))

('Final parameters:', 0.54193175, 0.20898542)
('True parameters: ', 0.5417657651097033, 0.20898520692791359)
